# Important Modules

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.stats.mstats import winsorize
import scipy.stats as stats


In [2]:

# Define the directory where raw CSV files are stored
raw_folder_path = '/Users/sujanbharadwaj/Library/Mobile Documents/com~apple~CloudDocs/Documents/Ontology_PCA_Project/data/raw/Raw data ESG data 2025Feb'  # Update this if needed

# Define the directory where processed files should be saved
processed_folder_path = "/Users/sujanbharadwaj/Library/Mobile Documents/com~apple~CloudDocs/Documents/Ontology_PCA_Project/data/processed"

# Ensure the processed folder exists
os.makedirs(processed_folder_path, exist_ok=True)

# List of ESG raw data files
csv_files = [
    "esg_raw_data_on_corporations_myu_5.csv",
    "esg_raw_data_on_corporations_myu_2.csv",
    "esg_raw_data_on_corporations_1.csv",
    "esg_raw_data_on_corporations_myu_4.csv",
    "esg_raw_data_on_corporations_myu_3.csv",
    "esg_raw_data_on_corporations_7.csv",
    "esg_raw_data_on_corporations_6.csv"
]

# Initialize empty lists for E, S, and G dataframes
e_dataframes = []
s_dataframes = []
g_dataframes = []

# Loop through each file and read the data
for filename in csv_files:
    file_path = os.path.join(raw_folder_path, filename)

    try:
        # Read CSV file with correct delimiter
        df = pd.read_csv(file_path, delimiter='|', encoding='utf-8')

        # Ensure 'pillar' column exists before filtering
        if 'pillar' in df.columns:
            e_dataframes.append(df[df['pillar'] == 'E'])
            s_dataframes.append(df[df['pillar'] == 'S'])
            g_dataframes.append(df[df['pillar'] == 'G'])

            print(f"Processed {filename}: E({len(e_dataframes[-1])}), S({len(s_dataframes[-1])}), G({len(g_dataframes[-1])})")
        else:
            print(f"Skipping {filename} - 'pillar' column missing.")

    except Exception as e:
        print(f"Error processing {filename}: {e}")

# Combine all files into one per category
df_e = pd.concat(e_dataframes, ignore_index=True)
df_s = pd.concat(s_dataframes, ignore_index=True)
df_g = pd.concat(g_dataframes, ignore_index=True)

# Save the new CSV files in the processed directory
df_e.to_csv(os.path.join(processed_folder_path, "esg_environmental_data.csv"), index=False, sep='|')
df_s.to_csv(os.path.join(processed_folder_path, "esg_social_data.csv"), index=False, sep='|')
df_g.to_csv(os.path.join(processed_folder_path, "esg_governance_data.csv"), index=False, sep='|')

print("CSV files saved successfully in the processed folder:")
print(f"- Environmental Data: {len(df_e)} rows")
print(f"- Social Data: {len(df_s)} rows")
print(f"- Governance Data: {len(df_g)} rows")


Processed esg_raw_data_on_corporations_myu_5.csv: E(396333), S(51959), G(74546)
Processed esg_raw_data_on_corporations_myu_2.csv: E(894601), S(196542), G(231277)
Processed esg_raw_data_on_corporations_1.csv: E(1269834), S(768008), G(726410)
Processed esg_raw_data_on_corporations_myu_4.csv: E(624877), S(74975), G(112890)
Processed esg_raw_data_on_corporations_myu_3.csv: E(767791), S(100980), G(155926)
Processed esg_raw_data_on_corporations_7.csv: E(50502), S(14844), G(21025)
Processed esg_raw_data_on_corporations_6.csv: E(190863), S(25985), G(40755)
CSV files saved successfully in the processed folder:
- Environmental Data: 4194801 rows
- Social Data: 1233293 rows
- Governance Data: 1362829 rows


In [3]:
# Define file paths
processed_folder_path = "/Users/sujanbharadwaj/Library/Mobile Documents/com~apple~CloudDocs/Documents/Ontology_PCA_Project/data/processed"
metric_summary_path = "/Users/sujanbharadwaj/Library/Mobile Documents/com~apple~CloudDocs/Documents/Ontology_PCA_Project/data/raw/metric_summary.csv"
industry_file_path = "/Users/sujanbharadwaj/Library/Mobile Documents/com~apple~CloudDocs/Documents/Ontology_PCA_Project/data/raw/industry.csv"  # Industry file

# Load the Metric Summary file
metric_summary = pd.read_csv(metric_summary_path)

# Ensure the required columns exist in the metric summary
if 'metric_name' not in metric_summary.columns or 'category' not in metric_summary.columns:
    raise ValueError("Metric Summary file must contain 'metric_name' and 'category' columns.")

# Load the Industry file
industry_data = pd.read_csv(industry_file_path)

# Rename columns for consistency
industry_data.rename(columns={'Industry': 'industry'}, inplace=True)

# Ensure 'perm_id' and 'industry' columns exist after renaming
if 'perm_id' not in industry_data.columns or 'industry' not in industry_data.columns:
    raise ValueError("Industry file must contain 'perm_id' and 'industry' columns.")

# Load processed ESG files
e_file = os.path.join(processed_folder_path, "esg_environmental_data.csv")
s_file = os.path.join(processed_folder_path, "esg_social_data.csv")
g_file = os.path.join(processed_folder_path, "esg_governance_data.csv")

df_e = pd.read_csv(e_file, delimiter='|', encoding='utf-8')
df_s = pd.read_csv(s_file, delimiter='|', encoding='utf-8')
df_g = pd.read_csv(g_file, delimiter='|', encoding='utf-8')

# Merge industry data into ESG datasets using 'perm_id'
df_e = df_e.merge(industry_data[['perm_id', 'industry']], on='perm_id', how='left')
df_s = df_s.merge(industry_data[['perm_id', 'industry']], on='perm_id', how='left')
df_g = df_g.merge(industry_data[['perm_id', 'industry']], on='perm_id', how='left')

# Split the datasets using the Metric Summary

# Environmental (E)
e_risk_metrics = metric_summary[metric_summary['category'] == 'Environmental Risk']['metric_name'].tolist()
e_opportunity_metrics = metric_summary[metric_summary['category'] == 'Environmental Opportunity']['metric_name'].tolist()

df_e_risk = df_e[df_e['metric_name'].isin(e_risk_metrics)]
df_e_opportunity = df_e[df_e['metric_name'].isin(e_opportunity_metrics)]

# Social (S)
s_risk_metrics = metric_summary[metric_summary['category'] == 'Social Risk']['metric_name'].tolist()
s_opportunity_metrics = metric_summary[metric_summary['category'] == 'Social Opportunity']['metric_name'].tolist()

df_s_risk = df_s[df_s['metric_name'].isin(s_risk_metrics)]
df_s_opportunity = df_s[df_s['metric_name'].isin(s_opportunity_metrics)]

# Governance (G)
g_risk_metrics = metric_summary[metric_summary['category'] == 'Governance Risk']['metric_name'].tolist()
g_opportunity_metrics = metric_summary[metric_summary['category'] == 'Governance Opportunity']['metric_name'].tolist()

df_g_risk = df_g[df_g['metric_name'].isin(g_risk_metrics)]
df_g_opportunity = df_g[df_g['metric_name'].isin(g_opportunity_metrics)]

# Save the final split files in the processed directory
df_e_risk.to_csv(os.path.join(processed_folder_path, "esg_environmental_risk.csv"), index=False, sep='|')
df_e_opportunity.to_csv(os.path.join(processed_folder_path, "esg_environmental_opportunity.csv"), index=False, sep='|')

df_s_risk.to_csv(os.path.join(processed_folder_path, "esg_social_risk.csv"), index=False, sep='|')
df_s_opportunity.to_csv(os.path.join(processed_folder_path, "esg_social_opportunity.csv"), index=False, sep='|')

df_g_risk.to_csv(os.path.join(processed_folder_path, "esg_governance_risk.csv"), index=False, sep='|')
df_g_opportunity.to_csv(os.path.join(processed_folder_path, "esg_governance_opportunity.csv"), index=False, sep='|')

# Print results
print("Splitting complete! Files saved successfully with Industry information (matched by perm_id) in the processed folder:")
print(f"- Environmental Risk: {len(df_e_risk)} rows")
print(f"- Environmental Opportunity: {len(df_e_opportunity)} rows")
print(f"- Social Risk: {len(df_s_risk)} rows")
print(f"- Social Opportunity: {len(df_s_opportunity)} rows")
print(f"- Governance Risk: {len(df_g_risk)} rows")
print(f"- Governance Opportunity: {len(df_g_opportunity)} rows")

Splitting complete! Files saved successfully with Industry information (matched by perm_id) in the processed folder:
- Environmental Risk: 3196816 rows
- Environmental Opportunity: 752505 rows
- Social Risk: 738425 rows
- Social Opportunity: 494868 rows
- Governance Risk: 257120 rows
- Governance Opportunity: 499455 rows


In [9]:
# Define file paths
processed_folder_path = "/Users/sujanbharadwaj/Library/Mobile Documents/com~apple~CloudDocs/Documents/Ontology_PCA_Project/data/processed"

# List of files to clean
files_to_clean = [
    "esg_environmental_risk.csv",
    "esg_environmental_opportunity.csv",
    "esg_social_risk.csv",
    "esg_social_opportunity.csv",
    "esg_governance_risk.csv",
    "esg_governance_opportunity.csv"
]

# Process each file
for filename in files_to_clean:
    file_path = os.path.join(processed_folder_path, filename)

    # Load the dataset
    df = pd.read_csv(file_path, delimiter='|', encoding='utf-8')

    # Extract year from 'metric_year' column
    if 'metric_year' in df.columns:
        df['year'] = pd.to_datetime(df['metric_year'], errors='coerce').dt.year  # Extract year
        df.drop(columns=['metric_year'], inplace=True)  # Remove original column after extraction

    # Drop columns that are completely empty (100% missing)
    df_cleaned = df.dropna(axis=1, how='all')

    # Save the cleaned file with `_cleaned` suffix
    cleaned_file_path = os.path.join(processed_folder_path, filename.replace('.csv', '_cleaned.csv'))
    df_cleaned.to_csv(cleaned_file_path, index=False, sep='|')

    print(f"✅ Cleaned file saved: {cleaned_file_path} (Rows: {len(df_cleaned)})")

print("\n🎯 Cleaning process complete! All cleaned files are now stored in the processed folder.")

✅ Cleaned file saved: /Users/sujanbharadwaj/Library/Mobile Documents/com~apple~CloudDocs/Documents/Ontology_PCA_Project/data/processed/esg_environmental_risk_cleaned.csv (Rows: 3196816)
✅ Cleaned file saved: /Users/sujanbharadwaj/Library/Mobile Documents/com~apple~CloudDocs/Documents/Ontology_PCA_Project/data/processed/esg_environmental_opportunity_cleaned.csv (Rows: 752505)
✅ Cleaned file saved: /Users/sujanbharadwaj/Library/Mobile Documents/com~apple~CloudDocs/Documents/Ontology_PCA_Project/data/processed/esg_social_risk_cleaned.csv (Rows: 738425)
✅ Cleaned file saved: /Users/sujanbharadwaj/Library/Mobile Documents/com~apple~CloudDocs/Documents/Ontology_PCA_Project/data/processed/esg_social_opportunity_cleaned.csv (Rows: 494868)
✅ Cleaned file saved: /Users/sujanbharadwaj/Library/Mobile Documents/com~apple~CloudDocs/Documents/Ontology_PCA_Project/data/processed/esg_governance_risk_cleaned.csv (Rows: 257120)
✅ Cleaned file saved: /Users/sujanbharadwaj/Library/Mobile Documents/com~appl

In [14]:
# Define file path
file_path = "/Users/sujanbharadwaj/Library/Mobile Documents/com~apple~CloudDocs/Documents/Ontology_PCA_Project/data/processed/esg_environmental_opportunity_cleaned.csv"

# Read the CSV file using the correct delimiter '|'
df_env_oppr_cleaned = pd.read_csv(file_path, delimiter='|', encoding='utf-8')

# Display the first few rows
print(df_env_oppr_cleaned.head())

      company_name     perm_id data_type  disclosure  \
0  Landsbankinn hf  5000632996       int  CALCULATED   
1  Landsbankinn hf  5000632996       int  CALCULATED   
2  Landsbankinn hf  5000632996     float    REPORTED   
3  Landsbankinn hf  5000632996       int  CALCULATED   
4  Landsbankinn hf  5000632996     float   ESTIMATED   

                                  metric_description  \
0  Does the company have a policy to improve its ...   
1  Does the company develop products or technolog...   
2  Total waste that is generated by the company a...   
3  Does the company develop products and services...   
4  Total energy consumed by a company within its ...   

                     metric_name metric_unit  metric_value  \
0        POLICY_WATER_EFFICIENCY      Yes/No          0.00   
1             WATER_TECHNOLOGIES      Yes/No          0.00   
2                 WASTE_RECYCLED        Tons         86.17   
3  SUSTAINABLE_BUILDING_PRODUCTS      Yes/No          0.00   
4               